In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing
from statsmodels.regression.linear_model import OLS

import matplotlib.colors as col
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import datetime
from pathlib import Path  
import random

# Scikit-Learn models:

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import catboost as cbt
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split






# LSTM:

import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping
# from keras.utils import np_utils
from keras.utils import to_categorical
from keras.layers import LSTM


# ARIMA Model:

import statsmodels.tsa.api as smt
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse


import pickle
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)


df_s = pd.read_excel("data/Argent_AFO_46045_SS_ship.xlsx")

In [2]:
def load_data(file_name):
    return pd.read_csv(file_name)


df_s = df_s[["Created", "Sku", "AdjustmentQty"]]
df_s.rename(columns={"Created": "Date", "AdjustmentQty": "ShippedQty"}, inplace=True)


print("min", df_s["Date"].min())
print("max", df_s["Date"].max())

min 1/11/2024 1:43:40 PM
max 9/8/2023 7:36:06 AM


In [4]:
print(df_s)

                       Date           Sku  ShippedQty
0      4/27/2023 6:58:37 AM  AFO 46045 SS           1
1      4/27/2023 7:10:36 AM  AFO 46045 SS           1
2      4/27/2023 7:10:36 AM  AFO 46045 SS           1
3      4/28/2023 9:30:39 AM  AFO 46045 SS           1
4      5/1/2023 10:47:34 AM  AFO 46045 SS           1
...                     ...           ...         ...
11082  11/6/2023 7:17:41 AM  AFO 46045 SS           1
11083  11/6/2023 7:17:41 AM  AFO 46045 SS           1
11084  11/6/2023 7:17:41 AM  AFO 46045 SS           1
11085  11/6/2023 7:17:41 AM  AFO 46045 SS           1
11086  11/6/2023 7:17:41 AM  AFO 46045 SS           1

[11087 rows x 3 columns]


In [5]:
# sales_data = load_data('data/train.csv')
# df_s = sales_data.copy()
# df_s

In [6]:
# test_data = load_data('data/test.csv')
# df_t = test_data.copy()
# df_t

In [7]:
# print("min", df_s["date"].min())
# print("max",dsf df_t["date"].max())
# # print(913000/45000)

In [ ]:
def sales_aggregation(data, time="monthly"):
    data = data.copy()
    
    if time == "monthly":
        data["date"] = pd.to_datetime(data["date"]).dt.to_period("M").astype(str)
    elif time == "yearly":
        data["date"] = pd.to_datetime(data["date"]).dt.to_period("Y").astype(str)
    elif time == "weekly":
        data["date"] = pd.to_datetime(data["date"]).dt.to_period("W").astype(str)
    elif time == "daily":
        data["date"] = pd.to_datetime(data["date"]).dt.date  # Keeps full date
    else:
        raise ValueError("Invalid time period. Choose from 'daily', 'weekly', 'monthly', or 'yearly'.")
    
    # Sum sales per selected time period
    data = data.groupby("date")["sales"].sum().reset_index()
    
    # Convert date column back to datetime format (except for weekly)
    if time in ["monthly", "yearly", "daily"]:
        data["date"] = pd.to_datetime(data["date"])

    return data



# Example usage
m_df = sales_aggregation(df_s, "monthly")
y_df = sales_aggregation(df_s, "yearly")
w_df = sales_aggregation(df_s, "weekly")
d_df = sales_aggregation(df_s, "daily")


layout = (2, 2)

plot1 = plt.subplot2grid(layout, (0 ,0))
plot2 = plt.subplot2grid(layout, (0 ,1))
plot3 = plt.subplot2grid(layout, (1 ,0))
plot4 = plt.subplot2grid(layout, (1 ,1))

years = y_df['sales'].plot(kind = "bar",color = 'mediumblue', label="Sales",ax=plot1, figsize=(12,5))
months = m_df['sales'].plot(kind = "bar",color = 'mediumblue', label="Sales",ax=plot2, figsize=(12,5))
# months = m_df['sales'].plot(marker = 'o',color = 'darkorange', label="Sales", ax=plot2)
weeks = w_df['sales'].plot(marker = 'o',color = 'darkorange', label="Sales", ax=plot3)
days = d_df['sales'].plot(color = 'darkorange', label="Sales", ax=plot4)

years.set(xlabel = "Years",title = "Distribution of Sales Per Year")
months.set(xlabel = "Months", title = "Distribution of Sales Per Month")
weeks.set(xlabel = "Weeks", title = "Distribution of Sales Per Week")
days.set(xlabel = "Days", title = "Distribution of Sales Per Day")

sns.despine()
plt.tight_layout()

years.legend()
months.legend()
weeks.legend()
days.legend()

In [ ]:
m_df

In [ ]:
def time_plot(data, x_col, y_col, title):
    fig, ax = plt.subplots(figsize=(15,5))
    sns.lineplot(x=x_col, y=y_col, data=data, ax=ax, color='darkblue', label='Total Sales')
    
    # Calculating the yearly mean of sales
    s_mean = data.groupby(data[x_col].dt.year)[y_col].mean().reset_index()
    s_mean[x_col] = pd.to_datetime(s_mean[x_col], format='%Y')  # Adjust this if the format doesn't match

    # Plotting the yearly mean
    sns.lineplot(x=s_mean[x_col], y=s_mean[y_col], ax=ax, color='darkred', label='Yearly Average Sales')

    ax.set(title=title, xlabel=x_col, ylabel=y_col)
    ax.legend()
    plt.show()
    
def get_diff(data):
    """Calculate the difference in sales month over month:"""
    
    data['sales_diff'] = data.sales.diff()
    data = data.dropna()
    
    data.to_csv('./stationary_df.csv')
    
    return data
    
time_plot(m_df, 'date', 'sales', 'Distribution of Sales Per Mounth' )
stationary_df = get_diff(m_df)
time_plot(stationary_df, 'date', 'sales_diff', 
          'Monthly Sales After Diff Transformation')

### ARIMA Modeling

In [ ]:
# def build_arima_data(data):
    
#     da_data = data.set_index('date').drop('sales', axis=1)
#     da_data.dropna(axis=0)
#     da_data.to_csv('./arima_df.csv')    
#     return da_data

# datatime_df = build_arima_data(stationary_df)

# def plots_lag(data, lags=None):
#     """Convert dataframe to datetime index"""
#     dt_data = data.set_index('date').drop('sales', axis=1)
#     dt_data.dropna(axis=0)
    
    
#     law  = plt.subplot(122)
#     acf  = plt.subplot(221)
#     pacf = plt.subplot(223)
    
#     dt_data.plot(ax=law, figsize=(10, 5), color='orange')
#     # Plot the autocorrelation function:
#     smt.graphics.plot_acf(dt_data, lags=lags, ax=acf, color='mediumblue')
#     smt.graphics.plot_pacf(dt_data, lags=lags, ax=pacf, color='mediumblue')
    
#     # Will also adjust spacing between subplots to minimize the overlaps:
#     plt.tight_layout()

# plots_lag(stationary_df, lags=24);

### Regressive Modeling

In [ ]:
model_df = stationary_df
print(model_df.head(10))

In [ ]:
print(model_df.columns)
print(model_df.shape)

In [ ]:
edwfrgethry

In [ ]:
def train_test_split_and_scale(data, train_size=0.8):
    """
    Splits the dataset into train and test sets, removes 'sales' and 'date' columns,
    and scales the data using MinMaxScaler.
    
    Parameters:
        data (DataFrame): Input dataset with a target column.
        train_size (float): Proportion of data to be used for training (default is 0.8).
    
    Returns:
        X_train, y_train, X_test, y_test, scaler: Scaled train and test sets with the scaler object.
    """
    # Ensure 'sales' and 'date' exist before dropping
    columns_to_drop = [col for col in ['sales', 'date'] if col in data.columns]
    data = data.drop(columns=columns_to_drop, errors='ignore')

    # Ensure data has at least 2 columns (features + target)
    if data.shape[1] < 2:
        raise ValueError("Dataset must have at least 2 columns (features + target).")

    # Train-test split
    train_index = int(len(data) * train_size)  # 80% train, 20% test
    train_set, test_set = data.iloc[:train_index].values, data.iloc[train_index:].values

    # Apply MinMaxScaler
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler.fit(train_set)  # Fit only on training data

    # Scale datasets
    train_set_scaled = scaler.transform(train_set)
    test_set_scaled = scaler.transform(test_set)

    # Feature-target split
    X_train, y_train = train_set_scaled[:, 1:], train_set_scaled[:, 0].ravel()
    X_test, y_test = test_set_scaled[:, 1:], test_set_scaled[:, 0].ravel()

    return X_train, y_train, X_test, y_test, scaler

# Example usage
X_train, y_train, X_test, y_test, scaler_object = train_test_split_and_scale(model_df)

# Print shapes
print(f"Shape of X Train: {X_train.shape}")
print(f"Shape of y Train: {y_train.shape}")
print(f"Shape of X Test: {X_test.shape}")
print(f"Shape of y Test: {y_test.shape}")

In [ ]:
import numpy as np

def train_test_split(data, train_size=0.8):
    data = data.drop(['sales', 'date'], axis=1)
    
    train_index = int(len(data) * train_size)  # 80% for training
    train, test = data[:train_index].values, data[train_index:].values
    
    return train, test

train, test = train_test_split(model_df)
print(f"Shape of Train: {train.shape}\nShape of Test: {test.shape}")

In [ ]:
test

### Scale Data

In [ ]:
def scale_data(train_set,test_set):

    
    # Apply Min Max Scaler:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(train_set)
    
    # Reshape training set:
    train_set = train_set.reshape(train_set.shape[0],
                                  train_set.shape[1])
    train_set_scaled = scaler.transform(train_set)
    
    # Reshape test set:
    test_set = test_set.reshape(test_set.shape[0], 
                                test_set.shape[1])
    test_set_scaled = scaler.transform(test_set)
    
    X_train, y_train = train_set_scaled[:, 1:], train_set_scaled[:, 0:1].ravel() # returns the array, flattened!
    X_test, y_test = test_set_scaled[:, 1:], test_set_scaled[:, 0:1].ravel()
    
    return X_train, y_train, X_test, y_test, scaler


X_train, y_train, X_test, y_test, scaler_object = scale_data(train, test)
print(f"Shape of X Train: {X_train.shape}\nShape of y Train: {y_train.shape}\nShape of X Test: {X_test.shape}\nShape of y Test: {y_test.shape}")

### Reverse Scaling

In [ ]:
def re_scaling(y_pred, x_test, scaler_obj, lstm=False):
    """For visualizing and comparing results, undoes the scaling effect on predictions."""
   # y_pred: model predictions
   # x_test: features from the test set used for predictions
   # scaler_obj: the scaler objects used for min-max scaling
   # lstm: indicate if the model run is the lstm. If True, additional transformation occurs 
    
    # Reshape y_pred:
    y_pred = y_pred.reshape(y_pred.shape[0],
                            1,
                            1)

    if not lstm:
        x_test = x_test.reshape(x_test.shape[0],
                                1, 
                                x_test.shape[1])

    # Rebuild test set for inverse transform:
    pred_test_set = []
    for index in range(0, len(y_pred)):
        pred_test_set.append(np.concatenate([y_pred[index], 
                                             x_test[index]],
                                             axis=1) )

    # Reshape pred_test_set:
    pred_test_set = np.array(pred_test_set)
    pred_test_set = pred_test_set.reshape(pred_test_set.shape[0],
                                          pred_test_set.shape[2])

    # Inverse transform:
    pred_test_set_inverted = scaler_obj.inverse_transform(pred_test_set)

    return pred_test_set_inverted

In [ ]:
def prediction_df(unscale_predictions, origin_df):
    """Generates a dataframe that shows the predicted sales for each month
    for plotting results."""
    
    # unscale_predictions: the model predictions that do not have min-max or other scaling applied
    # origin_df: the original monthly sales dataframe
    
    # Create dataframe that shows the predicted sales:
    result_list = []
    sales_dates = list(origin_df[-13:].date)
    act_sales = list(origin_df[-13:].sales)

    for index in range(0, len(unscale_predictions)):
        result_dict = {}
        result_dict['pred_value'] = int(unscale_predictions[index][0] + act_sales[index])
        result_dict['date'] = sales_dates[index + 1]
        result_list.append(result_dict)

    df_result = pd.DataFrame(result_list)

    return df_result

In [ ]:
model_scores = {}

def get_scores(unscale_df, origin_df, model_name):
    """Prints the root mean squared error, mean absolute error, and r2 scores
    for each model. Saves all results in a model_scores dictionary for
    comparison."""

    
    rmse = np.sqrt(mean_squared_error(origin_df.sales[-12:], 
                                      unscale_df.pred_value[-12:]))
    
    mae = mean_absolute_error(origin_df.sales[-12:], 
                              unscale_df.pred_value[-12:])
    
    r2 = r2_score(origin_df.sales[-12:], 
                  unscale_df.pred_value[-12:])
    
    model_scores[model_name] = [rmse, mae, r2]

    print(f"RMSE: {rmse}\nMAE: {mae}\nR2 Score: {r2}")

In [ ]:
def plot_results(results, origin_df, model_name):
    fig, ax = plt.subplots(figsize=(15,5))
    sns.lineplot(x='date', y='sales', data=origin_df, ax=ax, label='Original', color='blue')
    sns.lineplot(x='date', y='pred_value', data=results, ax=ax, label='Predicted', color='red')
    
    ax.set(xlabel="Date", ylabel="Sales", title=f"{model_name} Sales Forecasting Prediction")
    ax.legend()
    plt.show()
    
    ax.legend(loc='best')
    
    filepath = Path('./model_output/{model_name}_forecasting.svg')  
    filepath.parent.mkdir(parents=True, exist_ok=True) 
    plt.savefig(f'./model_output/{model_name}_forecasting.svg')

In [ ]:
def regressive_model(train_data, test_data, model, model_name):
    """Runs regressive models in SKlearn framework. First calls scale_data
    to split into X and y and scale the data. Then fits and predicts. Finally,
    predictions are unscaled, scores are printed, and results are plotted and
    saved."""
    
    # Split into X & y and scale data:
    X_train, y_train, X_test, y_test, scaler_object = scale_data(train_data,
                                                                 test_data)

    # Run sklearn models:
    mod = model
    mod.fit(X_train, y_train)
    predictions = mod.predict(X_test) # y_pred=predictions

    # Undo scaling to compare predictions against original data:
    origin_df = m_df
    unscaled = re_scaling(predictions, X_test, scaler_object) # unscaled_predictions
    unscaled_df = prediction_df(unscaled, origin_df)

    # Print scores and plot results:
    get_scores(unscaled_df, origin_df, model_name)
    plot_results(unscaled_df, origin_df, model_name)

In [ ]:
regressive_model(train, test, LinearRegression(), 'LinearRegression')